In [21]:
import cv2
import numpy as np
import matplotlib as plt
bright = cv2.imread('cube1.jpg')
dark = cv2.imread('cube8.jpg')

In [3]:
#observation : significant perceptual non-uniformity
#mixing of chrominance (color related information) and luminance (intensity related information) data

#Lab color-space
#L-lightness(intensity) ; independent of color info and encodes brightness only
#a-color component ranging from green to magenta ; encode color
#b-color component ranging from blue to yellow ; encode color

# it has the following properties
# perceptually uniform color space which approximates how we perceive color
# Independent of device ( capturing or displaying)
# Used extensively in Adobe Photoshop
# Is related to the RGB color space by a complex transformation equation

In [4]:
#python
brightLAB = cv2.cvtColor(bright, cv2.COLOR_BGR2LAB)
darkLAB = cv2.cvtColor(dark, cv2.COLOR_BGR2LAB)

In [5]:
#observation
# Similar observations as LAB can be made for Intensity and color components with regard to Illumination changes.
# Perceptual difference between Red and Orange is less even in the outdoor image as compared to LAB.
# White has undergone change in all 3 components.

#HSV color space
# H – Hue ( Dominant Wavelength ).
# S – Saturation ( Purity / shades of the color ).
# V – Value ( Intensity ).

# Best thing is that it uses only one channel to describe color (H), making it very intuitive to specify color.
# Device dependent.

In [14]:
# Thr YCrCb Color - space derived from the RGB color space and has the following three components
# Y – Luminance or Luma component obtained from RGB after gamma correction.
# Cr = R – Y ( how far is the red component from Luma ).
# Cb = B – Y ( how far is the blue component from Luma ).

# Separates the luminance and chrominance components into different channels.
# Mostly used in compression ( of Cr and Cb components ) for TV Transmission.
# Device dependent.


In [15]:
#python
brightYCB = cv2.cvtColor(bright, cv2.COLOR_BGR2YCrCb)
darkYCB = cv2.cvtColor(dark, cv2.COLOR_BGR2YCrCb)

In [6]:
brightHSV = cv2.cvtColor(bright, cv2.COLOR_BGR2HSV)
darkHSV = cv2.cvtColor(dark, cv2.COLOR_BGR2HSV)

In [7]:
# observation
# The H Component is very similar in both the images which indicates the color information is intact even under illumination changes.
# The S component is also very similar in both images.
# The V Component captures the amount of light falling on it thus it changes due to illumination changes.
# There is drastic difference between the values of the red piece of outdoor and Indoor image. This is because Hue is represented as a circle and red is at the starting angle. So, it may take values between [300, 360] and again [0, 60].

In [8]:
# how to use these color spaces for segmentation
# the simplest way : lets first try to use them to detect the green color from the cube.

#step1 : get the color values for a particular color
#step2: applying threshold for segmentation

In [18]:
#python
bgr = [40,158,16]
thresh = 40

# adjusting the color by subtract thresh from bgr which means the color value of one pixel
minBGR = np.array([bgr[0]-thresh, bgr[1]-thresh, bgr[2]-thresh])
maxBGR = np.array([bgr[0]-thresh, bgr[1]-thresh, bgr[2]-thresh])

maskBGR = cv2.inRange(bright, minBGR, maxBGR) # inRange for finding mask of green pixel
resultBGR = cv2.bitwise_and(bright, bright, mask = maskBGR) #maskBGR : 특정 이미지 표시, bright 부분만!
#bitwise_and : to get the green pixels from the image using the mask

#for converting one pixel to another color space, we first need to convert 1D array to a 3D array
#Convert 1D array to 3D, then convert it to HSV and take the first element
# this will be same as shown in the above figure [65,229,158]
hsv = cv2.cvtColor( np.uint8([[bgr]] ), cv2.COLOR_BGR2HSV)[0][0]#bgr값을 numpy 배열로 변환 -> hsv

minHSV = np.array([hsv[0]-thresh, hsv[1]-thresh, hsv[2]-thresh])
maxHSV = np.array([hsv[0]+thresh, hsv[1]+thresh, hsv[2]+thresh])

maskHSV = cv2.inRange(brightHSV, minHSV, maxHSV) # mask : pick certain area
#inRange function is to mask image!
resultHSV = cv2.bitwise_and(brightHSV, brightHSV, mask=maskHSV)

#convert 1D array to 3D, then convert it to YCrCb and take the first element
ycb = cv2.cvtColor( np.uint8([[bgr]] ), cv2.COLOR_BGR2YCrCb)[0][0]
 
minYCB = np.array([ycb[0] - thresh, ycb[1] - thresh, ycb[2] - thresh])
maxYCB = np.array([ycb[0] + thresh, ycb[1] + thresh, ycb[2] + thresh])
 
maskYCB = cv2.inRange(brightYCB, minYCB, maxYCB)
resultYCB = cv2.bitwise_and(brightYCB, brightYCB, mask = maskYCB)

#convert 1D array to 3D, then convert it to LAB and tke the first element
lab = cv2.cvtColor( np.uint8([[bgr]] ), cv2.COLOR_BGR2LAB)[0][0]

minLAB = np.array([lab[0] - thresh, lab[1] - thresh, lab[2] - thresh])
maxLAB = np.array([lab[0] + thresh, lab[1] + thresh, lab[2] + thresh])

maskLAB = cv2.inRange(brightLAB, minLAB, maxLAB)
resultLAB = cv2.bitwise_and(brightLAB, brightLAB, mask = maskLAB)
 
cv2.imshow("Result BGR", resultBGR)
cv2.imshow("Result HSV", resultHSV)
cv2.imshow("Result YCB", resultYCB)
cv2.imshow("Output LAB", resultLAB)


In [20]:
# Same data analysis for a better solution
# step 1 : data collection
# step 2 ; compute the density plot

#we will first load all images of blue or yellow pieces
B = np.array([])
G = np.array([])
R = np.array([])
im = cv2.imread(fi)

#seperate the channels and create and array for each channel by appending the values from each image
b = im[:,:,0] #모든 행, 열 선택 후 세 번째 차원에서 첫번째 채널만 선택하여 추출된 배열 또는 이미지 / 0번째 채널 = 파란색 채널
b = b.reshape(b.shape[0]*b.shape[1]) # 이미지의 높이(행) * 이미지의 너비(열) = 이미지의 전체 픽셀 수
g = im[:,:,1]
g = g.reshape(g.shape[0]*g.shape[1])
r = im[:,:,2]
r = r.reshape(r.shape[0]*r.shape[1])
B = np.append(B,b)
G = np.append(G,g)
R = np.append(R,r)

#use histogram plot from matplotlib to plot the 2D histogram
nbins = 10
plt.hist2d(B,G,bins=nbins, norm=LogNorm())
plt.xlabel('B')
plt.ylabel('G')
plt.xlim([0,255])
plt.ylim([0,255])

NameError: name 'fi' is not defined

In [ ]:
#Ideally, we want to work with a color space with the most compact / concentrated density plot for color channels.
#The density plots for RGB blow up drastically. This means that the variation in the values of the channels is very high and fixing a threshold is a big problem. Fixing a higher range will detect colors which are similar to the desired color ( False Positives ) and lower range will not detect the desired color in different lighting ( False Negatives ).
#In HSV, since only the H component contains information about the absolute color. Thus, it becomes my first choice of color space since I can tweak just one knob ( H ) to specify a color as compared to 2 knobs in YCrCb ( Cr and Cb ) and LAB ( A and B ).
#Comparing the plots of YCrCb and LAB shows a higher level of compactness in case of LAB. So, next best choice for me becomes the LAB color space.